In [1]:
import pandas as pd

import tensorflow as tf

import matplotlib.pyplot as plt

## Importing the dataset 

In [2]:
insurance = pd.read_csv('https://raw.githubusercontent.com/stedy/Machine-Learning-with-R-datasets/master/insurance.csv')

In [3]:
insurance

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest,10600.54830
1334,18,female,31.920,0,no,northeast,2205.98080
1335,18,female,36.850,0,no,southeast,1629.83350
1336,21,female,25.800,0,no,southwest,2007.94500


## Getting the dummies (Which is one hot encoding - First try this and then try normalized (Normalized might not work since the test dataset might be normalized) 

In [4]:
my_data = pd.get_dummies(insurance,drop_first=True)

In [5]:
my_data1= pd.get_dummies(insurance)

In [6]:
my_data

,age,bmi,children,charges,sex_male,smoker_yes,region_northwest,region_southeast,region_southwest
0,19,27.900,0,16884.92400,0,1,0,0,1
1,18,33.770,1,1725.55230,1,0,0,1,0
2,28,33.000,3,4449.46200,1,0,0,1,0
3,33,22.705,0,21984.47061,1,0,1,0,0
4,32,28.880,0,3866.85520,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...
1333,50,30.970,3,10600.54830,1,0,1,0,0
1334,18,31.920,0,2205.98080,0,0,0,0,0
1335,18,36.850,0,1629.83350,0,0,0,1,0
1336,21,25.800,0,2007.94500,0,0,0,0,1


## Creating the X and y values 

In [7]:
# Create X and Y values
X= my_data.drop('charges', axis = 1)
y = my_data['charges']

## Splitting the variables 

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=42)

## Model_checkpoint - callback

In [9]:
import os 
def create_model_checkpoint(model_name , save_path = 'model_experiments'):
    return tf.keras.callbacks.ModelCheckpoint(filepath = os.path.join(save_path, model_name),
                                             verbosity = 0, # Not print the output,
                                             save_best_only = True)

## Creating the model (By keeping the model checkpoint) -- Need to keep the validation for checkpoint

In [10]:
# Set random seed

tf.random.set_seed(42)

# Create the model

model = tf.keras.Sequential([
    tf.keras.layers.Dense(100,activation  = 'relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(1)
])


# Compile the model

model.compile(loss = tf.keras.losses.mae,
             optimizer = tf.keras.optimizers.Adam(learning_rate= 0.01),
              metrics= ['mae'])

#Creating a learning reduce on plateu callback

rpc = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.1,
    patience=10,
    verbose=0,
    mode="auto",
    min_delta=0.0001,
    cooldown=0,
    min_lr=0,
)

# Fit the model

history =  model.fit(X_train,y_train,epochs = 150, validation_data = (X_test, y_test),callbacks = [create_model_checkpoint(model_name='dense.h5'), rpc])

Epoch 1/150
34/34 [==============================] - 1s 11ms/step - loss: 13083.9482 - mae: 13083.9482 - val_loss: 12229.7402 - val_mae: 12229.7402 - lr: 0.0100
Epoch 2/150
34/34 [==============================] - 0s 3ms/step - loss: 11659.4551 - mae: 11659.4551 - val_loss: 10012.2812 - val_mae: 10012.2812 - lr: 0.0100
Epoch 3/150
34/34 [==============================] - 0s 3ms/step - loss: 9054.3057 - mae: 9054.3057 - val_loss: 7820.0996 - val_mae: 7820.0996 - lr: 0.0100
Epoch 4/150
34/34 [==============================] - 0s 3ms/step - loss: 7528.3853 - mae: 7528.3853 - val_loss: 7567.9399 - val_mae: 7567.9399 - lr: 0.0100
Epoch 5/150
34/34 [==============================] - 0s 4ms/step - loss: 7421.9170 - mae: 7421.9170 - val_loss: 7520.9004 - val_mae: 7520.9004 - lr: 0.0100
Epoch 6/150
34/34 [==============================] - 0s 4ms/step - loss: 7430.1216 - mae: 7430.1216 - val_loss: 7473.5859 - val_mae: 7473.5859 - lr: 0.0100
Epoch 7/150
34/34 [==============================] - 0s

34/34 [==============================] - 0s 4ms/step - loss: 3933.8691 - mae: 3933.8691 - val_loss: 3500.7939 - val_mae: 3500.7939 - lr: 0.0100
Epoch 106/150
34/34 [==============================] - 0s 3ms/step - loss: 3941.3032 - mae: 3941.3032 - val_loss: 3507.6833 - val_mae: 3507.6833 - lr: 0.0100
Epoch 107/150
34/34 [==============================] - 0s 4ms/step - loss: 3940.8308 - mae: 3940.8308 - val_loss: 3488.4282 - val_mae: 3488.4282 - lr: 0.0100
Epoch 108/150
34/34 [==============================] - 0s 3ms/step - loss: 3982.1658 - mae: 3982.1658 - val_loss: 3491.0662 - val_mae: 3491.0662 - lr: 0.0100
Epoch 109/150
34/34 [==============================] - 0s 3ms/step - loss: 3936.3645 - mae: 3936.3645 - val_loss: 3527.9949 - val_mae: 3527.9949 - lr: 0.0100
Epoch 110/150
34/34 [==============================] - 0s 4ms/step - loss: 3872.3538 - mae: 3872.3538 - val_loss: 3459.0281 - val_mae: 3459.0281 - lr: 0.0100
Epoch 111/150
34/34 [==============================] - 0s 3ms/step

In [11]:
model.evaluate(X_test, y_test)

9/9 [==============================] - 0s 2ms/step - loss: 3323.1604 - mae: 3323.1604


[3323.160400390625, 3323.160400390625]

In [ ]:
3109.3633

## Loading the best model 

In [14]:
model1 = tf.keras.models.load_model('model_experiments/dense.h5')

In [15]:
model1.evaluate(X_test, y_test)

9/9 [==============================] - 0s 2ms/step - loss: 3315.8218 - mae: 3315.8218


[3315.82177734375, 3315.82177734375]

## Creating the graph 

In [68]:
import pandas as pd

a = pd.DataFrame(history.history)
a
c = a.loc[:,['Accuracy','val_Accuracy']].plot(title = 'Not_normalized')

,loss,mae,val_loss,val_mae
0,13083.948242,13083.948242,12229.740234,12229.740234
1,11659.455078,11659.455078,10012.281250,10012.281250
2,9054.305664,9054.305664,7820.099609,7820.099609
3,7528.385254,7528.385254,7567.939941,7567.939941
4,7421.916992,7421.916992,7520.900391,7520.900391
...,...,...,...,...
245,3647.941650,3647.941650,3101.636475,3101.636475
246,3668.247314,3668.247314,3181.893311,3181.893311
247,3645.516357,3645.516357,3113.838135,3113.838135
248,3669.131348,3669.131348,3120.349121,3120.349121


## Normalizing and onehot encoding the data

In [10]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split


# Create a column transformer

ct = make_column_transformer((MinMaxScaler(), ['age','bmi','children']),
                            (OneHotEncoder(handle_unknown='ignore'),['sex','smoker','region'])
                            )


#Create X and y

X = insurance_new.drop('charges',axis = 1)
y = insurance_new['charges']

# Train and test split

X_train , X_test , y_train , y_test = train_test_split(X,y , test_size = 0.2 , random_state = 42)


# Fit the model -- Which mean's only the training values are considered such as in
#min max scalar minimum and max values are taken to normalize everything.
#so only the training min and max values are taken into consideration in this step only values are 
#calulated but transormation is done in the transformation step

ct.fit(X_train)

# Transform the value

X_train_normal = ct.transform(X_train)

X_test_normal = ct.transform(X_test)

In [ ]:
##

In [13]:
X = my_data.drop('y', axis = 1), X = my_data.drop('ID', axis = 1)

In [24]:
#X = X.drop('y', axis = 1)

In [26]:
X = pd.get_dummies(X)

In [27]:
X

,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,...,X8_p,X8_q,X8_r,X8_s,X8_t,X8_u,X8_v,X8_w,X8_x,X8_y
0,0,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4204,0,0,0,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4205,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4206,0,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4207,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
